Origin-destination matrix estimation

Possibly useful for constructing P matrix: probability of going through edges when moving from A to B

- https://openrouteservice.org/services/  (e.g. nearest)
- Google Maps API, maybe Google Distance Matrix API, maybe Google Geotagging API
- https://www.opentripplanner.org/ (Helps: https://github.com/jeromemayaud/OpenTripPlanner-OD-matrix)
Directions API: Directions between multiple locations, Distance Matrix API: Travel times and distances between locations<br>
Routes API: Performance optimized versions of the Directions and Distance Matrix APIs<br>
Roads API: Snap-to-road functionality to accurately trace GPS breadcrumbs.  maybe Places API

"QNEAT3 OD Matrix Calculation": https://github.com/kahlulz/Olongapo-City-Hospital-Network-Analysis<br>
Spatial Analysis: QGIS is utilized to conduct spatial analysis. Hospital locations were mapped, and distance-based calculations were performed using QNEAT3 OD Matrix Calculation. <br>
https://gis.stackexchange.com/questions/403556/what-unit-is-represented-by-od-matix-costs-in-qneat3-within-qgis-3-6-noosa?newreg=400dd6f344ab480a959ecfc8002019b9 <br>
https://root676.github.io/OdMatrixAlgs.html <br>
https://www.qgistutorials.com/en/docs/3/origin_destination_matrix.html

In [ ]:
import map_functions

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

gdf = gpd.read_file("data/sources/Hungary_2022_static_road_data.geojson")

## P Matrix: paths with Google

In [ ]:
import json
key = json.loads(open('keys.json').read())[""]

In [ ]:
import googlemaps

gmaps = googlemaps.Client(key=key)
directions_result = gmaps.directions("Hegyeshalom", "Mosonszolnok", alternatives=True)
#Possible directions test
for i, route in enumerate(directions_result):
    print(f"Route {i+1}:")
    for step in route['legs'][0]['steps']:  
        print(step['html_instructions'])
    print("\n\n")

Route 1:
Head <b>southwest</b> on <b>Alkotmány u.</b>
Turn <b>left</b>
Turn <b>right</b> at <b>Orgona u.</b>
Turn <b>right</b>
Turn <b>left</b>
Continue onto <b>Szabadság út</b>
Turn <b>right</b> onto <b>Jókai u.</b><div style="font-size:0.9em">Destination will be on the right</div>



Route 2:
Head <b>northeast</b> on <b>Alkotmány u.</b> toward <b>Damjanich u.</b>
Turn <b>right</b> onto <b>Damjanich u.</b>
Turn <b>left</b> onto <b>Szolnoki u.</b>
Continue onto <b>Kertalja u.</b>
Turn <b>right</b> onto <b>Fő u.</b>/<wbr/><b>Route 1</b><div style="font-size:0.9em">Continue to follow Route 1</div>
At the roundabout, take the <b>3rd</b> exit and stay on <b>Route 1</b>
At the roundabout, take the <b>2nd</b> exit and stay on <b>Route 1</b>
Turn <b>right</b> onto <b>Szolnoki út</b>
Continue onto <b>Óvári u.</b>
Continue onto <b>Fő u.</b>
Turn <b>left</b> onto <b>Szabadság út</b>
Turn <b>right</b> onto <b>Jókai u.</b><div style="font-size:0.9em">Destination will be on the right</div>



Route